In [17]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import warnings
warnings.filterwarnings('ignore')

def preprocess_air_quality_data(file_path):
    # Load data
    df = pd.read_csv(file_path, low_memory=False)
    df.drop(columns=['City'], inplace=True)
    
    # Handle missing values
    numeric_columns = df.select_dtypes(include=['float64']).columns
    imputer = SimpleImputer(strategy='mean')
    df[numeric_columns] = imputer.fit_transform(df[numeric_columns])
    
    # Process datetime and create time features
    df['Datetime'] = pd.to_datetime(df['Datetime'], format='mixed', errors='coerce')
    df['Year'] = df['Datetime'].dt.year
    df['Month'] = df['Datetime'].dt.month
    df['Day'] = df['Datetime'].dt.day
    df['Hour'] = df['Datetime'].dt.hour
    
    # Categorize PM2.5 values
    bins = [0, 12, 35.4, 55.4, 150.4, 250.4, float('inf')]
    labels = ['Good', 'Moderate', 'Unhealthy for Sensitive', 'Unhealthy', 'Very Unhealthy', 'Hazardous']
    df['PM2.5_Category'] = pd.cut(df['PM2.5'], bins=bins, labels=labels)
    
    # Perform undersampling
    return df


# For each dataset
df = preprocess_air_quality_data('../Data/Delhi_data.csv')

# Print sizes to see the reduction
print("Original sizes:", len(df))

Original sizes: 48192


In [18]:
df.head()

,Datetime,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket,Year,Month,Day,Hour,PM2.5_Category
0,2015-01-01 01:00:00,454.58,935.180000,81.52,41.78,187.66,27.54,9.29,3.41,54.94,25.24,58.57,13.80,260.148048,NaN,2015,1,1,1,Hazardous
1,2015-01-01 02:00:00,440.44,233.114669,70.80,43.46,176.83,27.72,13.28,3.88,50.53,23.10,49.37,15.63,260.148048,NaN,2015,1,1,2,Hazardous
2,2015-01-01 03:00:00,409.09,233.114669,132.46,41.19,141.02,28.94,29.67,2.83,19.33,19.04,38.94,17.18,260.148048,NaN,2015,1,1,3,Hazardous
3,2015-01-01 04:00:00,436.12,233.114669,84.78,39.55,102.84,29.30,21.76,4.33,20.08,13.99,27.53,16.82,260.148048,NaN,2015,1,1,4,Hazardous
4,2015-01-01 05:00:00,415.88,976.990000,60.24,37.41,80.12,30.84,26.19,6.17,16.00,11.14,21.99,14.29,260.148048,NaN,2015,1,1,5,Hazardous


In [19]:
df.describe()

,Datetime,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,Year,Month,Day,Hour
count,48192,48192.000000,48192.000000,48192.000000,48192.000000,48192.000000,48192.000000,48192.000000,48192.000000,48192.000000,48192.000000,48192.000000,48192.000000,48192.000000,48192.000000,48192.000000,48192.000000,48192.000000
mean,2017-10-01 00:30:00,117.665396,233.114669,39.005350,50.763704,58.587961,42.128140,1.979092,15.927299,51.312347,3.546031,17.187459,1.443237,260.148048,2017.271518,6.249626,15.723108,11.500000
min,2015-01-01 01:00:00,0.050000,2.990000,0.050000,2.660000,0.000000,0.570000,0.000000,0.020000,0.060000,0.000000,0.000000,0.000000,22.000000,2015.000000,1.000000,1.000000,0.000000
25%,2016-05-17 00:45:00,52.217500,128.917500,10.870000,30.290000,26.190000,30.040000,0.780000,9.840000,26.740000,1.500000,5.130000,0.190000,159.000000,2016.000000,3.000000,8.000000,5.750000
50%,2017-10-01 00:30:00,88.270000,214.085000,19.160000,44.480000,42.560000,37.790000,1.100000,13.960000,43.580000,2.590000,11.560000,1.443237,260.000000,2017.000000,6.000000,16.000000,11.500000
75%,2019-02-15 00:15:00,154.640000,306.142500,43.452500,64.580000,75.950000,48.580000,1.810000,18.850000,66.640000,4.580000,22.560000,1.443237,346.000000,2019.000000,9.000000,23.000000,17.250000
max,2020-07-01 00:00:00,938.500000,1000.000000,497.400000,337.820000,433.780000,485.520000,47.420000,187.050000,497.620000,93.340000,162.000000,158.830000,762.000000,2020.000000,12.000000,31.000000,23.000000
std,NaN,94.832311,138.263603,50.037844,27.998962,48.889830,20.165904,2.932460,9.842172,33.469571,3.224557,18.461292,3.153788,120.892140,1.600125,3.439819,8.796540,6.922258


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48192 entries, 0 to 48191
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Datetime        48192 non-null  datetime64[ns]
 1   PM2.5           48192 non-null  float64       
 2   PM10            48192 non-null  float64       
 3   NO              48192 non-null  float64       
 4   NO2             48192 non-null  float64       
 5   NOx             48192 non-null  float64       
 6   NH3             48192 non-null  float64       
 7   CO              48192 non-null  float64       
 8   SO2             48192 non-null  float64       
 9   O3              48192 non-null  float64       
 10  Benzene         48192 non-null  float64       
 11  Toluene         48192 non-null  float64       
 12  Xylene          48192 non-null  float64       
 13  AQI             48192 non-null  float64       
 14  AQI_Bucket      47694 non-null  object        
 15  Ye

In [21]:
df.isna().sum()

Datetime            0
PM2.5               0
PM10                0
NO                  0
NO2                 0
NOx                 0
NH3                 0
CO                  0
SO2                 0
O3                  0
Benzene             0
Toluene             0
Xylene              0
AQI                 0
AQI_Bucket        498
Year                0
Month               0
Day                 0
Hour                0
PM2.5_Category      0
dtype: int64

In [22]:
df_tmp = df.copy()

In [23]:
df_tmp.drop(['AQI_Bucket', 'Datetime'], axis=1, inplace=True)
df_tmp.head()

,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,Year,Month,Day,Hour,PM2.5_Category
0,454.58,935.180000,81.52,41.78,187.66,27.54,9.29,3.41,54.94,25.24,58.57,13.80,260.148048,2015,1,1,1,Hazardous
1,440.44,233.114669,70.80,43.46,176.83,27.72,13.28,3.88,50.53,23.10,49.37,15.63,260.148048,2015,1,1,2,Hazardous
2,409.09,233.114669,132.46,41.19,141.02,28.94,29.67,2.83,19.33,19.04,38.94,17.18,260.148048,2015,1,1,3,Hazardous
3,436.12,233.114669,84.78,39.55,102.84,29.30,21.76,4.33,20.08,13.99,27.53,16.82,260.148048,2015,1,1,4,Hazardous
4,415.88,976.990000,60.24,37.41,80.12,30.84,26.19,6.17,16.00,11.14,21.99,14.29,260.148048,2015,1,1,5,Hazardous


In [24]:
df_tmp['PM2.5_Category'] = df_tmp['PM2.5_Category'].astype("category").cat.as_ordered()

In [25]:
df_tmp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48192 entries, 0 to 48191
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   PM2.5           48192 non-null  float64 
 1   PM10            48192 non-null  float64 
 2   NO              48192 non-null  float64 
 3   NO2             48192 non-null  float64 
 4   NOx             48192 non-null  float64 
 5   NH3             48192 non-null  float64 
 6   CO              48192 non-null  float64 
 7   SO2             48192 non-null  float64 
 8   O3              48192 non-null  float64 
 9   Benzene         48192 non-null  float64 
 10  Toluene         48192 non-null  float64 
 11  Xylene          48192 non-null  float64 
 12  AQI             48192 non-null  float64 
 13  Year            48192 non-null  int32   
 14  Month           48192 non-null  int32   
 15  Day             48192 non-null  int32   
 16  Hour            48192 non-null  int32   
 17  PM2.5_Catego

In [26]:
df_tmp['PM2.5_Category'].cat.categories

Index(['Good', 'Moderate', 'Unhealthy for Sensitive', 'Unhealthy',
       'Very Unhealthy', 'Hazardous'],
      dtype='object')

In [27]:
df_tmp['PM2.5_Category'].cat.codes

0        5
1        5
2        5
3        5
4        5
        ..
48187    2
48188    2
48189    2
48190    2
48191    2
Length: 48192, dtype: int8

In [28]:
df_tmp.isnull().sum()/len(df_tmp)

PM2.5             0.0
PM10              0.0
NO                0.0
NO2               0.0
NOx               0.0
NH3               0.0
CO                0.0
SO2               0.0
O3                0.0
Benzene           0.0
Toluene           0.0
Xylene            0.0
AQI               0.0
Year              0.0
Month             0.0
Day               0.0
Hour              0.0
PM2.5_Category    0.0
dtype: float64

In [29]:
np.random.seed(42)
from sklearn.model_selection import train_test_split

X = df_tmp.drop("PM2.5", axis=1)
Y = df_tmp["PM2.5"]

X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, train_size=0.8)

len(X_train), len(X_valid)

X_train.shape, X_valid.shape

((38553, 17), (9639, 17))

In [30]:
for label, content in X_train.items():
    if pd.api.types.is_numeric_dtype(content):
        print(label)


PM10
NO
NO2
NOx
NH3
CO
SO2
O3
Benzene
Toluene
Xylene
AQI
Year
Month
Day
Hour


In [31]:
for label, content in X_train.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            print(label, "--> has", pd.isnull(content).sum(), "null values")

In [32]:
pd.Categorical(X_train["PM2.5_Category"]).codes

array([5, 2, 3, ..., 4, 4, 4], dtype=int8)

In [33]:
for label, content in X_train.items():
    if not pd.api.types.is_numeric_dtype(content):
        #turn category into number
        X_train[label] = pd.Categorical(content).codes + 1

In [34]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38553 entries, 180 to 15795
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PM10            38553 non-null  float64
 1   NO              38553 non-null  float64
 2   NO2             38553 non-null  float64
 3   NOx             38553 non-null  float64
 4   NH3             38553 non-null  float64
 5   CO              38553 non-null  float64
 6   SO2             38553 non-null  float64
 7   O3              38553 non-null  float64
 8   Benzene         38553 non-null  float64
 9   Toluene         38553 non-null  float64
 10  Xylene          38553 non-null  float64
 11  AQI             38553 non-null  float64
 12  Year            38553 non-null  int32  
 13  Month           38553 non-null  int32  
 14  Day             38553 non-null  int32  
 15  Hour            38553 non-null  int32  
 16  PM2.5_Category  38553 non-null  int8   
dtypes: float64(12), int32(4), int8(1)


In [35]:
for label, content in X_valid.items():
    if not pd.api.types.is_numeric_dtype(content):
        X_valid[label] = pd.Categorical(content).codes + 1

In [36]:
X_train.shape, X_valid.shape

((38553, 17), (9639, 17))

In [37]:
%%time
# Lets build a machine learning model
from sklearn.ensemble import RandomForestRegressor

CPU times: total: 0 ns
Wall time: 0 ns


In [38]:
np.random.seed(42)
from sklearn.metrics import mean_squared_log_error, mean_absolute_error, r2_score

def rmsle(y_test, y_preds):
    """
    calculate root mean square log error between prediction and true labels    
    """

    return np.sqrt(mean_squared_log_error(y_test, y_preds))

# create a function to evaluate model on a few different levels

def show_scores(model):
    train_preds = model.predict(X_train)
    val_preds = model.predict(X_valid)
    scores = {"Training MAE" : mean_absolute_error(Y_train,train_preds),
              "valid MAE" : mean_absolute_error(Y_valid, val_preds),
              "Training RMSLE" : rmsle(Y_train,train_preds),
              "valid RMSLE" : rmsle(Y_valid, val_preds),
              "Training R^2": r2_score(Y_train,train_preds),
              "valid R^2" : r2_score(Y_valid, val_preds)}
    
    return scores

In [39]:
model = RandomForestRegressor(n_jobs=-1, random_state=42)

In [40]:
%%time
model.fit(X_train, Y_train)

CPU times: total: 1min 27s
Wall time: 8.21 s


RandomForestRegressor(n_jobs=-1, random_state=42)

In [41]:
show_scores(model)

{'Training MAE': 3.9061771865032346,
 'valid MAE': 10.469884213211952,
 'Training RMSLE': 0.050252285005295626,
 'valid RMSLE': 0.13216314819080724,
 'Training R^2': 0.9949301870006582,
 'valid R^2': 0.9648405553899854}

In [42]:
%%time

# most ideal parameter 

ideal_model = RandomForestRegressor(n_estimators=90,
                                    min_samples_leaf=18,
                                    min_samples_split=3,
                                    max_features=0.5,
                                    n_jobs=-1,
                                    max_samples=None,
                                    random_state=42)  # random state so that our reult is reproducable

# FIT THE MODEL

ideal_model.fit(X_train, Y_train)

CPU times: total: 22.3 s
Wall time: 2.33 s


RandomForestRegressor(max_features=0.5, min_samples_leaf=18,
                      min_samples_split=3, n_estimators=90, n_jobs=-1,
                      random_state=42)

In [43]:
show_scores(ideal_model)

{'Training MAE': 10.831483610201603,
 'valid MAE': 12.297265398990035,
 'Training RMSLE': 0.13075651695402132,
 'valid RMSLE': 0.14972904307758964,
 'Training R^2': 0.9608882891293515,
 'valid R^2': 0.950249635062444}

# Linear Regression

In [44]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X_train,Y_train)

LinearRegression()

In [45]:
y_pred = linreg.predict(X_valid)

In [46]:
from sklearn.model_selection import cross_val_score
print('Cross_val_score', cross_val_score(linreg, X_train, Y_train, cv=10, scoring="r2").mean())
print('r2_score', r2_score(Y_valid, y_pred))
print("valid MAE ", mean_absolute_error(Y_valid, y_pred))

Cross_val_score 0.8090043399275737
r2_score 0.8125717858739933
valid MAE  26.5682042048491
